In [1]:
import streamlit as st
import pandas as pd
import duckdb
import plotly.express as px
from ydata_profiling import ProfileReport
from streamlit_pandas_profiling import st_profile_report

c:\Users\zeesh\anaconda3\envs\streamlit_new_2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = pd.read_excel("Financial Data Clean.xlsx")
df

,Account,business_unit,Currency,Year,Scenario,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,Sales,Software,USD,2012,Actuals,90924002.0,82606134.0,72780220.0,52943701.0,77528109.0,96384524.0,77345061.0,98290873.0,79879127.0,95373403.0,54887908.0,82703597.0
1,Cost of Goods Sold,Software,USD,2012,Actuals,-41623278.0,-40464347.0,-30806326.0,-21412962.0,-37047252.0,-44819597.0,-34847393.0,-47903350.0,-35880653.0,-44982115.0,-26929424.0,-34233473.0
2,Commissions Expense,Software,USD,2012,Actuals,-4454359.0,-3386032.0,-3389705.0,-2149257.0,-3168079.0,-4417624.0,-3386461.0,-4052846.0,-3418737.0,-4365527.0,-2455561.0,-3646726.0
3,Payroll Expense,Software,USD,2012,Actuals,-9901680.0,-9871172.0,-8459696.0,-6303408.0,-8493573.0,-11082494.0,-8081033.0,-11070018.0,-8410665.0,-10081727.0,-6300578.0,-9099438.0
4,Travel & Entertainment Expense,Software,USD,2012,Actuals,-951255.0,-838985.0,-872700.0,-624416.0,-919835.0,-1085296.0,-818602.0,-1040585.0,-803190.0,-1158623.0,-611335.0,-941542.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,Travel & Entertainment Expense,Hardware,USD,2023,Forecast,-295046.0,-288819.0,-270521.0,-299399.0,-259557.0,-306337.0,-302042.0,-335932.0,-352326.0,-202497.0,-209741.0,-306375.0
347,R&D Expense,Hardware,USD,2023,Forecast,-1063042.0,-1383994.0,-1005520.0,-1122436.0,-1105967.0,-1152280.0,-1172596.0,-1435420.0,-1381456.0,-865769.0,-855963.0,-1188799.0
348,Consulting Expense,Hardware,USD,2023,Forecast,-1440304.0,-1476449.0,-1202261.0,-1447697.0,-1227951.0,-1255275.0,-1424563.0,-1942571.0,-1593839.0,-1037651.0,-1097675.0,-1397647.0
349,Software/Hardware Expense,Hardware,USD,2023,Forecast,-1825596.0,-1941785.0,-1833335.0,-2389482.0,-1906406.0,-1777976.0,-2005933.0,-2661184.0,-2463031.0,-1455030.0,-1398480.0,-1954771.0


In [8]:
all_months= ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
sales_data = duckdb.sql(
    f"""
    WITH sales_data AS (
        SELECT
        Scenario,Year,{','.join(all_months)}
        FROM df
        WHERE Year='2023'
        AND Account='Sales'
        AND business_unit='Software'
        )
        UNPIVOT sales_data
        ON {','.join(all_months)}
        INTO 
            NAME month
            VALUE sales
        """
).df()
sales_data


,Scenario,Year,month,sales
0,Budget,2023,Jan,89862727.0
1,Budget,2023,Feb,99687807.0
2,Budget,2023,Mar,99378570.0
3,Budget,2023,Apr,55271910.0
4,Budget,2023,May,83758431.0
5,Budget,2023,Jun,51637163.0
6,Budget,2023,Jul,76348472.0
7,Budget,2023,Aug,66086281.0
8,Budget,2023,Sep,80081331.0
9,Budget,2023,Oct,83886832.0


In [9]:
sales_data= duckdb.sql(
    f"""
    WITH sales_data AS (
        UNPIVOT(
            SELECT Scenario,
            business_unit,
            {','.join(all_months)}
            FROM df 
            WHERE Year='2023'
            AND Account='Sales'
            )
        ON {','.join(all_months)}
        INTO 
            NAME month
            VALUE sales    
            ),
            aggregated_sales AS (
                SELECT
                Scenario, business_unit, SUM(sales) AS sales FROM sales_data
                GROUP BY Scenario, business_unit
            )
            SELECT * FROM aggregated_sales
            """
).df()   

sales_data


,Scenario,business_unit,sales
0,Budget,Software,9.687868e+08
1,Budget,Hardware,3.264234e+08
2,Forecast,Advertising,2.174004e+08
3,Budget,Advertising,2.397859e+08
4,Forecast,Software,8.881184e+08
5,Forecast,Hardware,3.047195e+08


In [10]:
sales_data = duckdb.sql(
    f"""
    WITH sales_data AS (
        UNPIVOT ( 
            SELECT 
                Account,Year,{','.join([f'ABS({month}) AS {month}' for month in all_months])}
                FROM df 
                WHERE Scenario='Actuals'
                AND Account!='Sales'
            ) 
        ON {','.join(all_months)}
        INTO
            NAME year
            VALUE sales
    ),

    aggregated_sales AS (
        SELECT
            Account,
            Year,
            SUM(sales) AS sales
        FROM sales_data
        GROUP BY Account, Year
    )
    
    SELECT * FROM aggregated_sales
"""
).df()
sales_data

,Account,Year,sales
0,Cost of Goods Sold,2013,653065582.0
1,Software/Hardware Expense,2016,124662567.0
2,Cost of Goods Sold,2017,698946667.0
3,Cost of Goods Sold,2019,643337258.0
4,Payroll Expense,2021,154765773.0
...,...,...,...
83,Cost of Goods Sold,2015,625700313.0
84,Cost of Goods Sold,2018,646623308.0
85,Commissions Expense,2020,56301265.0
86,Travel & Entertainment Expense,2020,14206558.0


In [13]:
import pandas as pd
from pandasql import sqldf

data = pd.DataFrame({'name': ['Alice', 'Bob', 'Charlie'], 'age': [25, 30, 20]})

query = "SELECT * FROM data WHERE age > 25"

result = sqldf(query)

result


,name,age
0,Bob,30
